# USE CASE - Build With AWS 

The Build with AWS app is a comprehensive guide to navigating Amazon Web Services (AWS), the leading global cloud provider with over 200 fully featured services. It addresses the challenge of selecting the right services by allowing users to input project details such as descriptions, libraries, and algorithms. Using this input, the app analyzes project requirements and generates a curated list of AWS services that align best with these needs.

The app provides step-by-step guidance on integrating these recommended services into projects, covering setup, configuration, and usage for effective implementation. Additionally, it offers access to AWS blogs and resources to aid project development. A built-in chatbot offers real-time support, allowing users to ask questions and receive assistance on AWS services and configurations.

We can effectively implement the solution using the Retrieval Augmented Generation (RAG) pattern. RAG involves retrieving data from sources external to the language model (non-parametric) and enhancing prompts by incorporating relevant retrieved data into the context. In this case, we are efficiently performing RAG on the knowledge base created using the console or SDK.

In this notebook, we'll query the knowledge base to retrieve the required number of document chunks through similarity search. These chunks will augment the prompt with relevant documents, forming the input query for Anthropic Claude V2 to generate responses.

### Prerequisite

Before answering questions, ensure that the following steps are completed for document processing and ingestion into the vector database:

1. **Load Documents into Knowledge Base**
   - Connect your S3 bucket as the data source to the knowledge base.

2. **Ingestion Process**
   - Knowledge bases will segment documents into smaller chunks based on the chosen strategy.
   - Generate embeddings from these chunks and store them in the associated vector store.

3. **Execution with Notebook 0_create_ingest_documents_test_kb.ipynb**
   - Utilize notebook `02_Create-Knowledge-Base.ipynb` to automate the ingestion process.

#### Walkthrough
We will leverage the Retrieve API provided by Knowledge Bases for Amazon Bedrock to convert user queries into embeddings, perform semantic search on the knowledge base, and retrieve relevant results. This API offers enhanced flexibility to customize workflows atop semantic search outcomes. The output from the Retrieve API includes retrieved text chunks, source data location type and URI, and relevance scores for each retrieval.

Subsequently, we will augment these text chunks with the original prompt. This augmented input will then be processed through the anthropic.claude-3-sonnet-20240229-v1:0 model using prompt engineering techniques tailored to your specific use case.

## Setup
To run this notebook you would need to install following packages.

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
%store -r kb_id
# kb_id = "<knowledge base id>" If you have already created knowledge base, comment the `store -r kb_id` and provide knowledge base id here.

To initialize the bedrock client and set up the anthropic.claude-3-sonnet-20240229-v1:0 language model for query completions using the RAG pattern alongside the knowledge base, follow these steps:

1. **Import Necessary Libraries**:
   - Import required libraries including langchain for bedrock model selection and llama index for managing service contexts.

2. **Initialize Service Context**:
   - Create a service context using llama index to store instances of language models (LLM) and embedding models. This context will be used later to evaluate responses from the Q&A application.

3. **Set anthropic.claude-3-sonnet-20240229-v1:0 as Language Model**:
   - Initialize anthropic.claude-3-sonnet-20240229-v1:0 as the chosen large language model (LLM). This model will facilitate query completions using the Retrieval Augmented Generation (RAG) pattern in conjunction with the retrieved text chunks obtained through the retrieve API.

In [ ]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

#### Retrieve API with foundation models from Amazon Bedrock

The "retrieve" function helps you find information by converting your questions into codes, searching for similar ideas. After searching, it brings back pieces of text that are similar to what you asked. Along with these text pieces, it also tells you where these ideas came from (like a web link) and how close they are to what you asked for (relevancy score). 

You can also choose how it searches using a setting called "overrideSearchType." If you let it decide on its own (the default setting), it picks the best way to find what you want. But if you set "overrideSearchType" to "HYBRID" or "SEMANTIC," you can tell it to use a specific way to search instead.

In [ ]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

Before querying responses from the initialized large language model (LLM), start by setting up or initializing your knowledge base ID.

After this setup, proceed to call the retrieve API. Provide the knowledge base ID, specify the number of results you want to receive, and include your query as parameters.

When you receive the results, each text chunk will come with a score. This score indicates how closely each chunk matches your query, helping you understand its relevance and correlation to the original question.

In [ ]:
query = "What are the products and services that could be used for Semi Structured Database application?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

#### Extract the text chunks from the retrieveAPI response

In [ ]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [ ]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

#### Prompt specific to the model to personalize responses

In [ ]:
prompt = f"""
Human: Analyze the requirement in query to briefly introduce and suggest relevant AWS Products, AWS services and Amazon services. Write a step-by-step usage of Relevant AWS Services based on the project. Suggest blogs and relevant resources from AWS to support the user to work with Relevant AWS Services and How I can use these AWS Services in my project?
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

#### Invoke foundation model from Amazon Bedrock

Using anthropic.claude-3-sonnet-20240229-v1:0 foundation model from Amazon Bedrock.

In [ ]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 1
        }  )


In [ ]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

pp.pprint(response_text)

#### LangChain Integration

1. Query the knowledge base to get the desired number of document chunks based on similarity search, 
2. Integrate it with LangChain retriever and use Anthropic Claude 3 Sonnet model for answering questions.

In [ ]:
# from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

llm = BedrockChat(model_id=modelId, 
                  client=bedrock_client)

In [ ]:
query = "Analyze the requirement in query to briefly introduce and suggest relevant AWS Products, AWS services and Amazon services. Write a step-by-step usage of Relevant AWS Services based on the project. Suggest blogs and relevant resources from AWS to support the user to work with Relevant AWS Services and How I can use these AWS Services in my project?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.get_relevant_documents(
        query=query
    )
pp.pprint(docs)

#### Prompt specific to the model to personalize responses

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: Analyze the requirement in query to briefly introduce and suggest relevant AWS Products, AWS services and Amazon services. Write a step-by-step usage of Relevant AWS Services based on the project. Suggest blogs and relevant resources from AWS to support the user to work with Relevant AWS Services and How I can use these AWS Services in my project?
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

#### Integrating the retriever and the LLM defined above with RetrievalQA Chain to build the Q&A application.

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [ ]:
answer = qa.invoke(query)
pp.pprint(answer)